In [55]:
import pandas as pd
import numpy as np

data = pd.read_csv('LMM_data1.csv')

times = data['t'].values
fwd0 = data['fwd'].values

#print(fwd0)
days = 252

periods = 5
Nums = periods 
sim = 5
fwd = np.zeros((sim, Nums, periods))
Dev = np.zeros((sim, Nums))
Timestep = np.zeros(periods + 1)
correl = np.zeros(periods + 1)
fwdvol = np.zeros(periods + 1)
Notional = np.zeros(periods + 1)
fixed_rate = np.zeros(periods + 1)
df = np.zeros((sim, Nums, periods + 1))
MtM = np.zeros((sim, periods))



for k in range(periods + 1):
    Timestep[k] = 1
    correl[k] = 1
    fwdvol[k] = 0.2
    fixed_rate[k]= 0.05
    Notional[k] = 1000000
    
#print(Timestep)


for i in range(sim):
    Orgsumfxd = 0
    Orgsumflt = 0
    sumfxd = 0
    sumflt = 0
    df[i, 0, 0] = 1
    j = 0
    for k in range(periods):
        fwd[i, 0, k] = fwd0[k]
        df[i, j, k + 1] = np.exp(-fwd[i, j, k] * Timestep[k]) * df[i, j, k]
        Orgsumfxd += Notional[k] * fixed_rate[k] * df[i, j, k + 1]
        Orgsumflt += Notional[k] * fwd[i, j, k] * df[i, j, k + 1]

    print(Orgsumfxd)
    print(Orgsumflt)
    
    MtM[i, 0] = Orgsumfxd -

    for j in range(1, periods):
        drift = 0    
        for k in range(j, periods):
            Z1 = np.random.normal(0,1)
            drift += (Timestep[k-1] * fwd[i, j-1, k] * fwdvol[j] * correl[k-1]) / (1 + fwd[i, j-1, k] * Timestep[k-1]) 
            fwd[i, j, k] = fwd[i, j-1, k] * np.exp((drift * fwdvol[k-1] - 0.5 * fwdvol[k-1] ** 2) * Timestep[k-1] + fwdvol[k-1] * Timestep[k-1] ** 0.5 * Z1)
            df[i, j, j] = 1
            df[i, j, k + 1] = np.exp(fwd[i, j, k] * Timestep[k]) * df[i, j, k]
            
            sumfxd += Timestep[k] * Notional[k] * fixed_rate[k] * df[i, j, k + 1]
            sumflt += Timestep[k] * Notional[k] * fwd[i, j, k] * df[i, j, k + 1]
            
        MtM[i, j] = sumfxd - sumflt
    
#print(df)
#print(fwd)

print(MtM)
#these shouldnt be 1's

224785.7515105168
214162.87647968406
224785.7515105168
214162.87647968406
224785.7515105168
214162.87647968406
224785.7515105168
214162.87647968406
224785.7515105168
214162.87647968406
[[      0.          -59498.6868187  -135440.72949191 -163792.52014299
  -167765.88362351]
 [      0.          -75229.3756129  -159032.04485572 -209329.73580525
  -263482.54619465]
 [      0.          -51816.3360273  -100907.42315448 -155612.13848686
  -197659.10466647]
 [      0.          -63523.96712625 -129697.31990071 -183445.37673472
  -228151.04204386]
 [      0.          -50719.44999918 -121221.8558039  -219549.7829679
  -249610.45132472]]


In [12]:
import os
current_path = os.getcwd()
print(current_path)

C:\Users\Laptop\pricing_models
